In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo",
               "Felipe Pizarro Marquez", ]
__email__ = ["jose@marroquin.cl", "fpizarro92@live.cl", ]
__status__ = "Development"

## Simulate Image Sets for Photogrammetry

With these functions, it is possible to generate **image sets to be used with a photogrammetry software**.

[blendjupyter.ipynb](blendjupyter.ipynb) is a shared Jupyter module &#171;written to set up and manipulate cameras (`'bpy.types.Camera'`), scenes (`'bpy.context.scene'`)&#187;, among other things.

This notebook **does not use** a Python kernel, [uses a Blender kernel](https://github.com/panzi/blender_ipython).

In [ ]:
import blendjupyter as blendj
import math
import os
import bpy

In [ ]:
def range_float(a, b, step):
    """Emulates the np.arange() function of the numpy Python module
    and returns the result as a list.
    
    With this function, IT IS NOT NECESSARY to install numpy for
    Blender."""
    l = list()
    if a > b:
        while a > b:
            l.append(a)
            a -= step
    else:
        if a < b:
            while a < b:
                l.append(a)
                a += step
    return l

In [ ]:
def walk_around(**kwargs):
    """Returns a list of pairs of tuples in which the first is a
    (x, y, z) coordinate and the second is the XYZ rotation angles
    for those coordinates, or a list of trios of tuples in which two
    are rotations angles and one is a location. If move ('str') is
    'OBJECT', the camera will be static in z-axis and y-axis and the
    object to be photographed will rotate from its location; else if
    move ('str') is 'CAM', the object will be static and the camera
    will move along a circular path in different heights and radius of
    a sphere.
    
    Example
    >>> l = walk_around(move='CAM')
    >>> l
    [((7.984359711335656, 0.0, 0.5000000000000012),
      (86.41667830152802, 0, 90)),
       ...,
     ((3.3777275704249043, -5.850395766102116, 4.2853034711634965),
      (57.61111886768535, 0, 390.0))]
    >>> (x, y, z), (rotx, roty, rotz) = l[0]
    >>> rotz
    90
    """
    r = kwargs.pop('r', 8)  # The radius in 'NONE' units.
    # Number of photos during movement in xy-plane.
    hsteps = kwargs.pop('hsteps', 16)
    # Number of xy-planes of movement.
    vsteps = kwargs.pop('vsteps', 4)
    # Starting angle for the camera respect to z-axis measured from
    # the xy-plane.
    z0 = kwargs.pop('z0', 30)
    # Ending angle for "[...]".
    ze = kwargs.pop('ze', 60)
    # In Blender, a simple way to find out the phase change angle is
    # placing the camera ('bpy.types.Camera') in the first (x, y, z)
    # coordinate and manually changing the Y rotation angle of
    # 'XYZ' rotation order.
    dphase = kwargs.pop('dphase', 90)
    # See option for move ('str') in the docstring under function
    # definition above.
    move = kwargs.pop('move', 'OBJECT')
    # Initial location of cam for moving type 'OBJECT' or 'SPIRAL'.
    loccam = kwargs.pop('loccam', (0, 0, 0))
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(walk_around.__name__, list(kwargs.keys())[-1]))
    # Phi is the angle measured from the z-Axis to the xy-plane.
    phi_init = 90 - z0
    phi_end = 90 - ze
    phi_step = abs(phi_init - phi_end) / float(vsteps)
    # Theta is the angle that rotates around the z-Axis.
    theta_step = 360 / float(hsteps)
    range_theta = range_float(0, 360, theta_step)
    range_theta = range_theta[:hsteps]
    l_coord = list()
    if move.lower() == 'cam':
        range_phi = range_float(phi_init, phi_end, phi_step)
    elif move.lower() == 'object':
        range_phi = range_float(90, phi_end, phi_step)
    range_phi = range_phi[:vsteps]
    for phi in range_phi:
        for theta in range_theta:
            if move.lower() == 'cam':
                # Parameterization of a sphere [1].
                #
                # [1] Carmen, M. (2007). Parametrizaciones. Retrieved
                # from: 
                # http://www.dm.uba.ar/materias/complementos_analisis_Mae/2007/2/parametrizaciones.pdf
                x = r * math.cos(math.radians(theta)) * math.sin(math.radians(phi))
                y = r * math.sin(math.radians(theta)) * math.sin(math.radians(phi))
                z = r * math.cos(math.radians(phi)) + loccam[2]
                rotx = phi
                roty = 0
                rotz = theta + dphase
                l_coord.append(((x, y, z), (rotx, roty, rotz)))
            elif move.lower() == 'object':
                x_cam = loccam[2] * math.cos(math.radians(phi))
                x_cam = loccam[0]
                roty_axis = phi
                rotz_mesh = theta
                l_coord.append(((0, roty_axis, 0),
                                (0, 0, rotz_mesh),
                                (x_cam, loccam[1], loccam[2])))
    return l_coord

In [ ]:
def point_shoot_cam(cam, sc, axis, mesh, l_loc_rot, **kwargs):
    """Locates the camera ('bpy.types.Camera') and the object to be
    photographed, and points the camera and shoots it.
    
    Returns the path route in which it will store the photos."""
    inrad = kwargs.pop('inrad', False)  # It is True if the rotation
                                        # angles are in radians.  
    path = blendj.find_out_dir(dirname='blend-phg-set-0001',
                               parentdir='scanner')
    move = kwargs.pop('move', 'OBJECT')
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(point_shoot_cam.__name__, list(kwargs.keys())[-1]))
    len_llocrot = len(l_loc_rot)
    for i in range(len_llocrot):
        if move.lower() == 'cam':
            loc, rot = l_loc_rot[i]
            cam.location = loc
            if not inrad:
                for j in range(3):  # 3 by each rotation.
                    cam.rotation_euler[j] = math.radians(rot[j])
        elif move.lower() == 'object':
            axis_rot, mesh_rot, cam_loc = l_loc_rot[i]
            if not inrad:
                for j in range(3):  # 3 by each rotation.
                    axis.rotation_euler[j] = math.radians(axis_rot[j])
                    mesh.rotation_euler[j] = math.radians(mesh_rot[j])
            cam.location = cam_loc
        blendj.shoot_cam(cam,
                         sc,
                         i,
                         len_llocrot,
                         path,
                         prefix='view_')
    return path

In [ ]:
def take_phg_photos(**kwargs):
    """Prepares a scene and a cam to capture photos of a mesh.
    
    Returns the route of the output folder."""
    s3path = kwargs.pop('s3path',
                        os.path.join(os.path.expanduser('~'),
                                     'super-scanner-software-s3'))
    # For sample files included with Super Scanner Software.
    name = kwargs.pop('name', 'phstudio_ArtemisStatue.blend')
    fileroute = kwargs.pop('fileroute',
                           os.path.join(s3path, 'blend-meshes', name))
    meshname = kwargs.pop('meshname', '3DWP ArtemisStatue Thingiverse')
    # Size for the render in (width, height) pixels.
    size = kwargs.pop('size', (2560, 1920))
    scale = kwargs.pop('scale', 1)
    gray = kwargs.pop('gray', False)
    # movetype ('str') can be 'OBJECT' or 'CAM'. See walk_around()
    # ('list').
    movetype = kwargs.pop('movetype', 'OBJECT')
    radius = kwargs.pop('radius', 12)
    z0angle = kwargs.pop('z0angle', 0)
    zeangle = kwargs.pop('zeangle', 10)
    zsteps = kwargs.pop('zsteps', 3)
    xysteps = kwargs.pop('xysteps', 12)
    camloc0 = kwargs.pop('camloc0', (2, 0, 9))
    transparent = kwargs.pop('transparent', False)
    extension = kwargs.pop('extension', 'JPEG')
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(take_phg_photos.__name__, list(kwargs.keys())[-1]))
    try:
        bpy.ops.wm.open_mainfile(filepath=fileroute)
        print('Opened:', fileroute)
    except IOError:
        print('No such file:', fileroute)
    # Get the axis ('EMPTY') associated to the mesh ('MESH') with the
    # name meshname ('str').
    axis, mesh = blendj.get_mesh(meshname)
    scene = blendj.set_up_sc(wpx=size[0], hpx=size[1],
                             scalepercent=scale * 100,
                             bw=gray, transp=transparent,
                             ext=extension)
    cam = blendj.get_camera()
    blendj.set_up_cam(cam, loc=camloc0)
    if movetype.lower() == 'cam':
        return point_shoot_cam(cam, scene, axis, mesh,
                               walk_around(move='CAM', r=radius,
                                           z0=z0angle, ze=zeangle,
                                           hsteps=zsteps,
                                           vsteps=xysteps,
                                           loccam=cam.location),
                               move=movetype)
    elif movetype.lower() == 'object':
        return point_shoot_cam(cam, scene, axis, mesh,
                               walk_around(loccam=cam.location,
                                           z0=z0angle, ze=zeangle,
                                           vsteps=zsteps,
                                           hsteps=xysteps),
                               move=movetype)